# Шаблон архитектуры системы управления данными "Lakehouse"

## Мотивация

### Хранилища Данных - Data Warehouse, DWH

**Хранилища данных** (**Data Warehouse**) исторически были призваны решить задачу формирования аналитической картины для бизнеса на базе собираемой информации. Аналитика позволяет бинесу принимать взвешанные решения и формировать стратегию развития компании. Хранилища данных предъявляют требования к формату входящих данных, что позволяет пользователям хранилища данных работать с понятными данными. Характерной чертой традицонных хранилищ данных (**Data Warehouse**) является неразрывная связь между вычислительными ресурсами и ресурсами для хранения данных.

С увеличением объема поступающей информации традицонный подход к формированию хранилища данных столкнулись с большими проблемами: неразрывность слоёв хранения и обработки данных диктовала бизнесу вертикальное масштабирование. Бизнес был вынужен закупать железо, способное обслуживать пиковые нагрузки. Таким образом, бизнес тратил много денег, хотя пиковые нагрузки могут и не возникать часто.

В дополнение к увеличению объема обрабатываемой информации альтернативные форматы данных, которые не всегда имеют чёткую структуру (видео, музыка, текст и т.д.), также начали играть большую роль в построении аналитической картины. К неструктурированным данным **хранилища данных** были не готовы абсолютно.

### Озёра Данных - Data Lake

Для решения накопившихся проблем были разработаны платформы аналитики данных **второго поколения**.

Основная идея заключается в том, что все поступающие данные начали складываться в **озёра данных** (**Data Lake**) - кластер из дешёвых жестких дисков. Данные в **озёре данных** хранятся открытых (open source) форматах, таких как Apache Parquet или ORC.

Apache Hadoop с HDFS популяризровал подход с озером данных. От требования к формату входящих данных (`schema-on-write`) удалось перейти к `schema-on-read` - только конечный потребитель знает какие данные ему нужны, и он сам является ответственным за извлечение этих данных из озера данных. Таким образом, `ETL` (**Extract Transform Load**) был заменен на `ELT` (**Extract Load Transform**): бизнес смог собирать всю доступную информацию, которая потом анализировалась различными подходами.

Архитектура **Озеро Данных + Хранилища Данных** является доминирующей в современном мире.

Отличительной особенностью озера данных является разделение слоёв хранения данных и обработки данных: бизнес может заказывать дополнительные процессорные мощности при необходимости.

### Ограничения современной архитектуры

В свою очередь озера данных (Data Lake) принесли свои проблемы связанные с качеством и управлением данных. При этом данные по прежнему необходимо доставлять в хранилища данных, к которым подключены витрины данных. Таким образом, все поступающие данные проходят два или три шага:

- E**L**T для попадания в озеро данных (Data Lake),
- ET**L** для попадания в хранилище данных (Warehouse),
- (опиционально) ETL для машинного обучения.

Современний бизнес также полагается на системы машинного обучения и искуственного интеллекта, для которых ни озёра данных (Data Lake), ни хранилища данных (Data Warehouse) не являются идеальным решением.

Среди проблем современной архитектуры организации данных (**Озеро Данных** + **Хранилища Данных**) можно выделить следующие категории:

- **Надежность**,
- **Свежесть данных**,
- **Ограниченные возможности расширенной аналитики**,
- **Итоговая стоимость владения данными**.

#### Надежность

Задача поддежрки озера данных и хранилища данных в консистентном состоянии является сложной и затратной. Перекачка данных из озера данных в хранилище данных несёт риски связанные со сбоем оборудования или ошибками программиста, что может выражаться в некоторой разнице между озером данных и хранилищем данных, а это ухудшает качество данных.

#### Свежесть данных

Самые новые данные всегда сначала в озеро данных, а уже потом в хранилище. Перекачка данных из озера данных в хранилище не может выполниться мнгновенно. Таким образом, современные аналитические системы уступают своим предшественникам (**Data Warehouse**) по качеству свежести данных.

#### Ограниченные возможности расширенной аналитики

Бизнес должен иметь возможность задавать вопросы относительно будущего на основании собранных данных (например, "Кому эффективнее всего предоставить скидки?").

Современные фреймворки организации машинного обучения PyTorch, TensorFlow, XGBoost имеют ограниченные возможности работы на базе хранилища данных. В отличии от привычных хранилищам запросов BI характера, которые работают с небольшим объемом информации, фреймворки машинного обучения требуют обработки огромных объемов данных.

Для подключения машинного обучения есть два варианта:

- выгрузить данные из хранилища данных в файлы, которые можно использовать при работе с машинным обучением (третий ETL шаг),
- работа с озером данных напрямую. Это отключает возможности хранилища данных по управлению данными:
    - ACID транзакции,
    - версионирование данных,
    - индексирование.

#### Итоговая стоимость владения данными

Одновременная поддержка озера данных и хранилища данных фактически увеличивает затраты на хранение данных вдвое: одни и те же данные лежат как в озере данных ("грязные"), так и в хранилище данных ("чистые"). При этом также в два раза увеличиваются риски утечки данных.

### Требования к организации системы управления данными

Таким образом, необходимо организовать данные в озере данных, которое отвечает следующим требованиям:

1. открытость форматов данных (Parquet, ORC) для хранения как защита от Vendor Lock-In,
2. производительность сопоставимая с хранилищами данных,
3. возможности управления атрибутами/признаками (feature) данных, сопоставимые с хранилищами данных,
4. прямой `I/O` доступ к данным для расширенной аналитики.

Шаблон архитектуры **Lakehouse** позволяет построить озеро данных, которое отвечает поставленным требованиям.

Функция `dump_parquet` скроет подробности сохранения датафрейма на диск как parquet:

# Шаблон Lakehouse

## Подготовка данных

Распаковать подготовленные архивы с данными:

In [ ]:
! cd /tmp && rm -rf steam taxi.parquet && unzip -o ~/work/data/steam.zip && unzip -o ~/work/data/taxi.zip

Файлы будут располагаться в HDFS, поэтому необходимо очистить файловую систему:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -rm -f -r /user/jovyan/data && \
hdfs dfs -mkdir -p /user/jovyan/data/steam && \
hdfs dfs -mkdir -p /user/jovyan/data/taxi

Загрузить локальные файлы в Apache Spark, разбить их на 4 партиции и положить в HDFS:

In [ ]:
def dump_parquet(df, num: int, loc: str):
    (df
        .repartition(num)
        .write
        .mode("overwrite")
        .parquet(loc)
    )

In [ ]:
def prepare_src_data(sc, data_path):
    sc.setJobDescription("Разбить датафреймы steam на 4 партиции")

    dump_parquet(spark.read.parquet("file:///tmp/steam/details.parquet"), 4, f"{data_path}/details")
    dump_parquet(spark.read.parquet("file:///tmp/steam/tags.parquet"), 4, f"{data_path}/steam/tags")
    dump_parquet(spark.read.parquet("file:///tmp/steam/games.parquet"), 4, f"{data_path}/steam/games")
    dump_parquet(spark.read.parquet("file:///tmp/taxi.parquet"), 4, f"{data_path}/taxi")

## Описание

Lakehouse предлагает организовать систему управления данными на базе следующих концепций:

- хранение данных в кластера из дешевых жестких дисков (HDFS, S3, GCS),
- система демонстрирует функции аналитической СУБД:
    - ACID транзакции,
    - версионирование данных,
    - аудит,
    - индексы,
    - кеширование,
    - оптимизатор запросов.

Таким образом, система построеная по шаблону Lakehouse заключает в себе комбинацию ключевых преимуществ озёр данных (Data Lake) и хранилищ данных (Data Warehouse):

- низкая стоимость хранения данных,
- открытые форматы данных (как в Data Lake),
- мощные возможности управления данными и оптимизации запросов (как в Data Warehouse).

## Концепции Lakehouse

Шаблон Lakehouse состоит из следующих концепций:

- все данные хранятся в блочном хранилище вроде HDFS или S3;
- все данные приводятся к стандартному формату, например, Parquet;
- данные могут быть организованы в виде логических таблиц;
- поверх слоя хранения данных реализован слой метаданных;
- слой метаданных может содержать:
    - статистику по файлам,
    - список файлов, относящихся к текущей версии таблицы.
- ACID транзакции, версионирование, индексация - манипуляции со слоем метаданных;
- клиенты могут читать файлы из HDFS или S3 напрямую.

### Реализация Lakehouse

Форматы [Delta Lake](https://delta.io/) (Databricks), [Apache Iceberg](https://iceberg.apache.org/) (Netflix) и [Apache Hudi](https://hudi.apache.org/) (Uber) реализуют концепции похожие на Lakehouse схожим образом:

- данные сохраняются в формате Parquet или ORC в блочное хранилище (HDFS, S3),
- сохраненные файлы неизменны,
- рядом с сохраненными данными создаеются файлы с метаданными,
- при внесении изменений в файлы, создаются новые файлы,
- метаданные отслеживают какие из имеющихся файлов относятся к текущей версии.

Формат Delta Lake был разработан в компании Databricks, которая развивает Apache Spark, поэтому построение системы управления данными на примере Lakehouse в практической части будет основываться на Delta Lake.

> **Delta Lake - основной формат для построения Lakehouse!**

## Ограничения Delta Lake

Один файл с метаданными относится к одному Parquet файлу с данными, поэтому, к сожалению, невозможно получить ACID транзакции, включающие множество файлов. Эта проблема известна и находится на контроле ответственных организаций.

Учитывая, что Databrics запускает примерно 50% (на 2021 год) всей нагрузки своих клиентов через Delta Lake, возможно отсутствие ACID транзакций на несколько файлов не является большой проблемой.

## Особенности Delta Lake

Вне зависимости от используемого блочного хранилища Delta Lake эффективно реализует оптимизации:

- пропуск файлов при сканировании,
- кластеризация строк.

### Анализ статистики для пропуска файлов при сканировании

Вне зависимости от используемого блочного хранилища, файл с метаданными хранит информацию, которая может использоваться оптимизатором запросов. В метаданных хранится статистика о максимальных и минимальных значениях колонок для каждого файла, что позволяет исключать файлы из сканирования.

### Пропуск файлов при сканировании на базе Bloom фильтра

Для каждого parquet файла строится Bloom фильтр, который также позволяет эффективно пропускать файлы, в которых точно нет требуемых данных.

### Кластеризация строк

Delta Lake при помощи Z-Order оптимизации определяет наилучшую организацию строк на диске, с учетом, что несколько строк будут наиболее часто считываться вместе.

## Delta Lake

Устанавливать пакет для работы с Delta Lake необязательно, т.к. чтением данных с диска, их обработкой и сохранением данных обратно на диск в формате Delta Lake занимаются воркеры, которые запускаются в JVM процессе. Таким образом, нужно добавить библиотки для работы с Delta Lake на `CLASSPATH` воркеров.

Подготовить `CLASSPATH` можно при помощи указания соответствующих значений в `spark.jars`, `spark.jars.packages`, `spark.jars.repositories`, а можно установить Python пакет `delta-lake` и воспользоваться утилитными функциями для этих целей:

In [ ]:
! pip install delta-spark==3.0.0

### Запуск Apache Spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col
from pyspark.sql import functions as F

In [ ]:
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [ ]:
builder = (
    SparkSession
        .builder
        .appName("Delta Demo")
        .master("local[4]")
        .config("spark.sql.warehouse.dir", "data/spark-warehouse")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

Функция [`configure_spark_with_delta_pip`](https://docs.delta.io/latest/api/python/index.html#delta.pip_utils.configure_spark_with_delta_pip) активирует возможности Delta Lake:

In [ ]:
print(configure_spark_with_delta_pip.__doc__)

In [ ]:
spark = configure_spark_with_delta_pip(builder) \
    .getOrCreate()

sc = spark.sparkContext

### Подготовить данные

In [ ]:
prepare_src_data(sc, "/user/jovyan/data")

Файлы загружены в HDFS:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data

## Delta таблицы

In [ ]:
my_first_table_path = 'data/my-first-delta-table'

### Создание Delta таблицы

Сохранение датафрейма в формате `delta` создает новую Delta таблицу:

In [ ]:
data = spark.range(0, 5)
(
    data
        .write
        .format("delta")
        .save(my_first_table_path)
)

### Загрузка Delta таблицы

Чтение данных из `delta` таблицы:

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

### Обновление Delta таблицы

Запись датафрейма в формате `delta` в режиме `overwrite` обновляет данные в `delta` таблице:

In [ ]:
df = spark.range(5, 10)
(
    df
        .write
        .format("delta")
        .mode("overwrite")
        .save(my_first_table_path)
)

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

### Частичное обновление Delta таблицы

Данные в таблице можно обновить по условию:

In [ ]:
deltaTable = DeltaTable.forPath(spark, my_first_table_path)

deltaTable.update(
    condition = F.expr("id % 2 == 0"),
    set = { "id": F.expr("id + 100") }
)

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

### Удаление данных Delta таблицы

Удалить данные в таблице можно можно обновить по условию:

In [ ]:
deltaTable = DeltaTable.forPath(spark, my_first_table_path)
deltaTable.delete(condition = F.expr("id % 2 == 0"))

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

### Слияние данных - Merge, Upsert

Данные могут быть записаны в таблицу по принципу слияния (merge, upsert), т.е. существующие данные заменяются, новые данные добавляются:

In [ ]:
df = (
    spark.read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

In [ ]:
deltaTable = DeltaTable.forPath(spark, my_first_table_path)

# New data to upsert (merge) 
newData = spark.range(7, 20) #.withColumnRenamed("id", "x")

deltaTable.alias("old").merge(
    source = newData.alias("new"),
    condition = col("old.id") == col("new.id")
  ).whenMatchedUpdate(set = 
    {
       "id": col("new.id")
    }
  ).whenNotMatchedInsert(condition=col("new.id").isNotNull(), values =
    {
        "id": col("new.id") 
    }
  ).whenNotMatchedBySourceDelete() \
  .execute()

Обратите внимание, что операция `execute` выполняется немедленно.

In [ ]:
newData.show()

In [ ]:
(
    deltaTable.history()
        .orderBy(col("version").desc())
        .limit(1)
        .show(vertical=True, truncate=False)
)

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

## Машина времени

Все файлы, загруженные в HDFS через Delta Lake, становятся неизменяемыми. Но как тогда выполняется обновление данных?

Delta создает новые файлы в директории таблицы и при помощи дополнительных файлов отслеживает, какие файлы составляют содержимое текущей версии таблицы.

Таким образом, Delta Lake поддерживает версионирование, а значит в любой момент можно вернуться к любой из предыдущих версий: файлы по прежнему находятся в директории таблицы. В качестве примера можно вернуться к самой первой версии:

In [ ]:
df = (
    spark.read
        .format("delta")
        .option("versionAsOf", 0)
        .load(my_first_table_path)
)

df.show()

Также можно сбросить состояние таблицы на момент одной из предыдущих версий:

In [ ]:
deltaTable = DeltaTable.forPath(spark, my_first_table_path)
deltaTable.history().show()

Сбросить таблицу до начального состояния:

In [ ]:
deltaTable.restoreToVersion(0)

Обратите внимание, что `restoreToVersion` выполняется немедленно.

При этом создается новая версия:

In [ ]:
deltaTable.history().show()

Содержимое таблицы:

In [ ]:
deltaTable.toDF().show()

## Конвертация Parquet в Delta Lake

Delta Lake работает поверх Parquet файлов, поэтому конвертация parquet в Delta Lake заключается в добавлении файлов с метаданными в директорию с Parquet файлами.

Состояние директории без файлов с метаданными Delta Lake:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

In [ ]:
taxi_path = '/user/jovyan/data/taxi'

In [ ]:
deltaTable = DeltaTable.convertToDelta(spark, f"parquet.`{taxi_path}`")

In [ ]:
print(DeltaTable.convertToDelta.__doc__)

Состояние директории после конвертации в Delta Lake:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

Директория `_delta_log` содержит метаданные, например схема:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -cat /user/jovyan/data/taxi/_delta_log/00000000000000000000.json | \
grep metaData | json_pp | \
grep schema | sed 's,\s*"schemaString" : ",,;s,"$,,;s,\\",",g' | json_pp > schema.json

Вывод команды сохранен в [schema.json](schema.json)

Или статистика по колонкам (`min`, `max`, `total`):

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -cat /user/jovyan/data/taxi/_delta_log/00000000000000000000.json | \
grep add |& head -n 1 | json_pp | \
grep stats | sed 's,\s*"stats" : ",,;s,"$,,;s,\\",",g' | json_pp > stats.json

Вывод команды сохранен в [stats.json](stats.json)

## Z-Order оптимизация

In [ ]:
deltaTable = DeltaTable.forPath(spark, taxi_path)

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

Оптимизация Z-Order позволяет организовать кластеризацию записей, т.е. сохранение похожих данных вместе. Например, если часто нужно получать агрегированные значения среди поездок в такси с разным числом пассажиров, то имеет смысл выполнить Z-Order оптимизацию по колонке `passenger_count`:

In [ ]:
df = deltaTable.optimize().executeZOrderBy("passenger_count")

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

Результатом операции [`executeZOrderBy`](https://docs.delta.io/latest/api/python/index.html#delta.tables.DeltaOptimizeBuilder.executeZOrderBy) будет датафрейм со статистикой по операции:

In [ ]:
df.printSchema()

In [ ]:
(
    df.select("metrics.zOrderStats.inputOtherFiles", "metrics.zOrderStats.mergedFiles")
        .select(
            col("inputOtherFiles.num").alias("input_num_files"), col("inputOtherFiles.size").alias("input_size"),
            col("mergedFiles.num").alias("merged_num_files"), col("mergedFiles.size").alias("merged_size")
        )
        .show(truncate=False)
)

Также статистику можно получить из истории:

In [ ]:
(
deltaTable.history()
    .select(
        "version",
        "timestamp",
        "operation",
        "operationMetrics.numRemovedFiles",
        "operationMetrics.numAddedFiles",
        "operationMetrics.numConvertedFiles")
    .show(truncate=False)
)

## Вакуумная очистка

In [ ]:
deltaTable = DeltaTable.forPath(spark, taxi_path)

Количество файлов таблицы может расти бесконтрольно, но не все из них нужны. Файлы, которые относятся к старым версиям можно удалять при помощи [`vacuum`](https://docs.delta.io/latest/api/python/index.html#delta.tables.DeltaTable.vacuum):

In [ ]:
import sys

try:
    deltaTable.vacuum(
        retentionHours=0 # Optional, the default value is 168
    )
except Exception as e:
    print(e, file=sys.stderr)

При этом нельзя удалять слишком свежие данные (моложе 168 часов, 7 дней), т.к. зачастую ошибка.

Если действительно необходимо удалить свежие данные, то необходимо снять проверку:

In [ ]:
spark.conf.set('spark.databricks.delta.retentionDurationCheck.enabled', False)

Текущий список файлов в директории:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

In [ ]:
deltaTable.vacuum(retentionHours=0)

_(Желательно возвращать проверку, т.к. она распространяется на все таблицы)_

In [ ]:
spark.conf.set('spark.databricks.delta.retentionDurationCheck.enabled', True)

Можно заметить, что теперь на файловой системе нет файлов для предыдущих версий:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

В истории по прежнему перечислены все версии:

In [ ]:
deltaTable.history().show()

Если запросить данные для версии, файлы которой удалены, то появится ошибка:

In [ ]:
import sys

try:
    (
    spark.read
        .format("delta")
        .option("versionAsOf", 0)
        .load(taxi_path)
        .show()
    )
except Exception as e:
    print(e.java_exception.getMessage(), file=sys.stderr)

In [ ]:
spark.stop()

# Построение Lakehouse архитектуры

In [ ]:
spark.stop()

## Пример системы для обработки

В качестве демонстрации построения платформы данных на базе шаблона Lakehouse будет рассмотрена система с компьютерными играми и отзывами.

### Доменная модель

Основные сущности системы:
- игры - games,
- пользователи - users,
- отзывы - reviews.

Набор игр список пользователей не меняется.

### Формат отзывов

Отзывы приходят в формате `json`.

Источники отзывов: Metacritic и Игромания.

Формат отзывов Metacritic:

- **id INT**: идентификатор пользователя,
- **app_id INT**: идентификатор игры,
- **review_date STRING**: дата отзыва в виде строки (`2022-01-01`),
- **review STRING**: текст отзыва,
- **score INT**: оценка игры по мнению пользователя (цело число в интервале `[0, 100]`: `81`,`95`, и т.д.),
- **user_id INT**: идентификатор пользователя.

Формат отзывов Игромания:

- **id INT**: идентификатор пользователя,
- **app_id INT**: идентификатор игры,
- **review_date_unix LONG**: дата отзыва в UNIX формате - число секунд с начала эпохи (`1970-01-01`),
- **review STRING**: текст отзыва,
- **rating FLOAT**: оценка игры по мнению пользователя (десятичное число в интервале `[0, 10]`: `7.3`,`5.2`, и т.д.),
- **user_id INT**: идентификатор пользователя.

Можно увидеть, что схемы входных данных отличаются:

- даты в разных форматах,
- оценка игры имеет разные названия,
- оценка игры имеет разную точность.

Следовательно, будет необходимо привести данные к единому формату.

### Поток данных - Workflow

Источниками данных для отзывов являются данные в формате json, расположенные в HDFS по пути `/user/jovyan/data/games/source`.

Поток данных выглядит следующим образом:

1. Для каждого входного файла запускается стрим, который генерирует записи с колонками:
    - `seen_time TIMESTAMP` - текущее время для индикации времени обработки входных данных в Spark,
    - `data STRING` - строки входного файла. Читаются как текст (десериализация в `json` не происходит).
1. Бронзовые таблицы являются приемниками стримов источников данных;
1. Каждая бронзовая таблица стартует стрим, который:
    - разбирает колонку `data` по json схеме,
    - добавляет колонку с именем источника,
    - указывает значение ID, котрое будет уникальным среди всех данных бронзовых таблиц.
1. Все стримы бронзовых таблиц записывают свои данные в **одну** серебрянную таблицу;
1. Для единственной серебрянной таблицы стартует стрим, который создает золотуют таблицу как таблицу фактов с колонками:
    - `user_id` - идентификатор пользователя,
    - `app_id` - идентификатор игры,
    - `review_id` - идентификатор отзыва.

Картинка ниже демонстрирует поток данных между слоями:


![](../imgs/spark-steam-reviews-worflow.drawio.png)

Цветовой индикацией отмечены поля одинаковые по смыслу, но отличающиеся по форме, которые необходимо унифицировать

Интересной возможностью Spark Structured Streaming является стриминг данных из директорий, именно на базе этой идеи поток данных и будет работать.

## Запуск приложения

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col
from pyspark.sql import functions as F

In [ ]:
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [ ]:
src_games_path = "data/steam/games"

base_path = "data/games"
source_path = base_path + "/source"

checkpoint_path = base_path + "/checkpoints"

In [ ]:
builder = (
    SparkSession
        .builder
        .appName("Delta Demo")
        .master("local[4]")
        .config("spark.sql.warehouse.dir", "data/spark-warehouse")
        .config("hive.metastore.uris", "thrift://hive:9083")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .enableHiveSupport()
)

spark = configure_spark_with_delta_pip(builder) \
    .getOrCreate()

sc = spark.sparkContext

Состояние Spark Catalog:

In [ ]:
spark.catalog.listTables()

## Подготовка данных

### Обновить директорию с данными

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -rm -r -f data/games

In [ ]:
prepare_src_data(sc, "data")

Файлы загружены в HDFS:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data

### Генерация синтетических отзывов

Конвертация `games` в Delta Lake:

In [ ]:
(
    spark
        .read
        .parquet(src_games_path)
        .write
        .format("delta")
        .mode("overwrite")
        .saveAsTable("games")
)

Утилитные методы для генерации отзывов:

In [ ]:
from random import randrange

def generate_reviews(
    ids_range,
    rating, date,
    max_users=100000
):
    games_df = spark.table("games")

    ids = [ F.lit(x) for x in ids_range ]
    
    return (games_df
        .withColumn("id", F.explode(F.array(ids)))
        .select(
            F.expr("app_id * 1000000 + id").alias("id"),
            "app_id",
            date,
            col("title").alias("review"),
            rating,
            F.floor(F.rand() * max_users).alias("user_id"),
        )
    )

def generate_reviews_metacritic(ids_range):
    rating=F.floor(F.rand() * 100).alias("score")
    date=F.expr("STRING(DATE_ADD(date_release, CAST(floor(rand() * 500) AS INT)))").alias("review_date")
    return generate_reviews(ids_range, rating, date)

def generate_reviews_igromania(ids_range):
    rating=F.round(F.rand() * 10, 1).alias("rating")
    date=F.expr("unix_timestamp() + floor(rand() * 2 * 12 * 30 * 24 * 60 * 60)").alias("review_date_unix")
    return generate_reviews(ids_range, rating, date)

Подготовка датарфеймов с отзывами:

In [ ]:
reviews_metacritic = generate_reviews_metacritic(ids_range=range(1, 10))
reviews_igromania = generate_reviews_igromania(ids_range=range(1, 5))

In [ ]:
reviews_metacritic.show(5, truncate=False)
reviews_igromania.show(5, truncate=False)

Функция `save_df` абстрагирует сохранение датафрейма на диск:

In [ ]:
def save_df(df, location, format="delta", mode="append"):
    (
    df
        .write
        .format(format)
        .mode(mode)
        .save(location)
    )

In [ ]:
save_df(reviews_metacritic, f"{source_path}/reviews_metacritic", format="json", mode='overwrite')
save_df(reviews_igromania, f"{source_path}/reviews_igromania", format="json", mode='overwrite')

Проверить, что нет дублей по ключу:

In [ ]:
def check_duplicates(location: str, format: str="delta"):
    return (
        spark.read
            .format(format)
            .option("path", location)
            .load()
            .groupBy("id")
            .count()
            .where("count > 1")
    )

In [ ]:
check_duplicates(f"{source_path}/reviews_metacritic", "json").count()

In [ ]:
check_duplicates(f"{source_path}/reviews_igromania", "json").count()

### Генерация синтетических пользователей

Пользователи будут иметь следующие атрибуты:

- идентификатор: `id`,
- имя пользователя: `username`,
- половая принадлежность: `gender`,
- дата рождения (date of birth): `dob`.

Всего будет подготовлено 90 тысяч пользователей

In [ ]:
users_df = (
    spark.range(0, 90000)
        .withColumn("username", F.concat(F.lit("user"), F.lit("_"), col("id")))
        .withColumn("gender", F.when(F.rand() > 0.6, "F").otherwise("M"))
        .withColumn("dob", F.expr("ADD_MONTHS(current_date(), -CAST(floor(rand() * 25 * 12) AS INT) -10 * 12)"))
)
users_df.show(5)

In [ ]:
users_df.write.format("delta").mode("overwrite").saveAsTable("users")

In [ ]:
spark.catalog.listTables()

## Уровни данных

Архитектура Lakehouse различает три уровня данных:

- **бронзовый уровень (Bronze Layer)**: данные, которые попали в Data Lake без модификации с добавлением метаданных (например, дата поступления строки);
- **серебрянный уровень (Silver Layer)**: очищенные бронзовые данные, имеющие определенную структуру;
- **золотой уровень (Golden Layer)**: данные, готовые, пригодные для запросов с витрин данных (Data Mart).

### Бронзовый уровень - Bronze Layer

In [ ]:
spark.sql("DROP TABLE IF EXISTS bronze_reviews_igromania")

In [ ]:
spark.sql("DROP TABLE IF EXISTS bronze_reviews_metacritic")

Бронзовые данные будут представлять собой таблциы с двумя колоками:

- `seen_time` дата поступления строки,
- `data` входные json данные в текстовом виде.

In [ ]:
def ingest_raw_reviews(source_path: str, checkpoint_dir: str, sink_table_name: str):
    raw_reviews_read_stream = (
        spark.readStream
                .format("text")
                .schema("data STRING")
                .option("maxFilesPerTrigger", 1)
                .option("checkpointLocation", checkpoint_dir)
                .load(source_path)
                .withColumn("seen_time", F.current_timestamp())
    )
    
    return (
        raw_reviews_read_stream
            .writeStream
            .queryName(f"Lakehouse: {source_path} -> {sink_table_name}")
            .format("delta")
            .option("checkpointLocation", checkpoint_dir)
            .outputMode("append")
            .trigger(processingTime='5 seconds')
            .toTable(sink_table_name)
    )

In [ ]:
bronze_reviews_igromania_stream = ingest_raw_reviews(
    source_path=f"{source_path}/reviews_igromania",
    checkpoint_dir=f"{checkpoint_path}/igromania",
    sink_table_name="bronze_reviews_igromania"
)

In [ ]:
bronze_reviews_metacritic_stream = ingest_raw_reviews(
    source_path=f"{source_path}/reviews_metacritic",
    checkpoint_dir=f"{checkpoint_path}/metacritic",
    sink_table_name="bronze_reviews_metacritic"
)

In [ ]:
spark.table("bronze_reviews_igromania").show(5, vertical=True, truncate=False)

In [ ]:
spark.table("bronze_reviews_metacritic").show(5, vertical=True, truncate=False)

In [ ]:
spark.table("bronze_reviews_igromania").count()

In [ ]:
spark.table("bronze_reviews_metacritic").count()

In [ ]:
spark.table("bronze_reviews_igromania").count()

In [ ]:
spark.table("bronze_reviews_metacritic").count()

In [ ]:
spark.table("bronze_reviews_igromania").select(F.count_distinct("seen_time")).show()

### Серебрянный уровень - Silver Layer

Серебрянный уровень содержит очищенные данные. В нашем случае, входные данные из бронзовых таблицы разбираются JSON парсером, и результат попадает в таблицу `silver_reviews`. В результате:

- две бронзовые таблицы `bronze_reviews_igromania` и `` сливаются в одну `silver_reviews` таблицу,
- имена и типы колонок унифицируются,
- значения `rating` масштабируются на интервал целых чисел `[0-100]`,
- каждая строка помечается из какого источника она пришла,
- каждая строка имеет уникальный идентификатор среди всех источников данных, вычисленный по формуле `ИМЯ_ИСТОЧНИКА + '_' + ID`.

In [ ]:
spark.sql("DROP TABLE IF EXiSTS silver_reviews")

In [ ]:
def ingest_bronze_reviews(bronze_table: str, checkpoint_dir: str, sink_table_name: str, json_parser, *cols):
    bronze_reviews_read_stream = (
        spark.readStream
                .format("delta")
                .option("maxFilesPerTrigger", 1)
                .option("checkpointLocation", checkpoint_dir)
                .table(bronze_table)
                .select("seen_time", json_parser)
                .select(*cols)
    )
    
    return (
        bronze_reviews_read_stream
            .writeStream
            .queryName(f"Lakehouse: {bronze_table} -> {sink_table_name}")
            .format("delta")
            .option("checkpointLocation", checkpoint_dir)
            .outputMode("append")
            .trigger(processingTime='5 seconds')
            .toTable(sink_table_name)
    )

In [ ]:
reviews_igromania.printSchema()

In [ ]:
igromania_json_parser = F.from_json(
    "data",
    """
        id INT,
        app_id INT,
        review_date_unix LONG,
        review STRING,
        rating DOUBLE,
        user_id LONG
    """
).alias("payload")

silver_reviews_from_igromania_stream = ingest_bronze_reviews(
    "bronze_reviews_igromania",            # `bronze_table` param
    f"{checkpoint_path}/igromania_bronze", # `checkpoint_dir` param
    "silver_reviews",                      # `sink_table_name` param
    igromania_json_parser,                 # `json_parser` param

    # columns to select after parsing the json:
    "seen_time",
    F.concat(F.lit("IGROMANIA"), col("payload.id")).alias("id"),
    "payload.app_id",
    F.from_unixtime("payload.review_date_unix").cast("TIMESTAMP").alias("review_ts"),
    "payload.review",
    F.floor(col("payload.rating") * 10).alias("rating"),
    "payload.user_id",
    F.lit("IGROMANIA").alias("source")
)

In [ ]:
spark.table("silver_reviews").count()

In [ ]:
reviews_metacritic.printSchema()

In [ ]:
metacritic_json_parser = F.from_json(
    "data",
    """
        id LONG,
        app_id INT,
        review_date STRING,
        review STRING,
        score INT,
        user_id LONG
    """
).alias("payload")

silver_reviews_from_metacritic_stream = ingest_bronze_reviews(
    "bronze_reviews_metacritic",            # `bronze_table` param
    f"{checkpoint_path}/metacritic_bronze", # `checkpoint_dir` param
    "silver_reviews",                       # `sink_table_name` param
    metacritic_json_parser,                 # `json_parser` param

    # columns to select after parsing the json:
    "seen_time",
    F.concat(F.lit("METACRITIC"), col("payload.id")).alias("id"),
    "payload.app_id",
    F.to_timestamp("payload.review_date").alias("review_ts"),
    "payload.review",
    F.floor(col("payload.score")).alias("rating"),
    "payload.user_id",
    F.lit("METACRITIC").alias("source")
)

In [ ]:
spark.table("silver_reviews").count()

In [ ]:
spark.table("silver_reviews").show(10, truncate=False)

### Золотой уровень - Golden Layer

Таблицы на золотом уровне находятся в формате готовом для витрин данных. Одним из самых распространенных форматов является схема "Звезда": таблица фактов + таблицы измерений.

Создадим таблицу фактов на базе серебрянной таблицы `silver_reviews` для следующего запроса:

In [ ]:
df = (
    spark.table("silver_reviews")
        .select(
            col("id").alias("review_id"),
            "app_id",
            "user_id"
        )
)
df.show(5, truncate=False)

In [ ]:
spark.sql("DROP TABLE IF EXISTS golden_review_facts")

In [ ]:
def ingest_silver_reviews(silver_table: str, checkpoint_dir: str, sink_table_name: str, *cols):
    silver_reviews_read_stream = (
        spark.readStream
                .format("delta")
                .option("maxFilesPerTrigger", 1)
                .option("checkpointLocation", checkpoint_dir)
                .table(silver_table)
                .select(*cols)
    )
    
    return (
        silver_reviews_read_stream
            .writeStream
            .queryName(f"Lakehouse: {silver_table} -> {sink_table_name}")
            .format("delta")
            .option("checkpointLocation", checkpoint_dir)
            .outputMode("append")
            .trigger(processingTime='5 seconds')
            .toTable(sink_table_name)
    )

In [ ]:
spark.table("silver_reviews").printSchema()

In [ ]:
golden_review_facts_stream = ingest_silver_reviews(
    "silver_reviews",                         # `silver_table` param
    f"{checkpoint_path}/review_facts_golden", # `checkpoint_dir` param
    "golden_review_facts",                    # `sink_table_name` param

    # columns for dimensions
    col("id").alias("review_id"),
    "app_id",
    "user_id"
)

In [ ]:
spark.table("golden_review_facts").count()

In [ ]:
spark.catalog.listTables()

При помощи таблицы фактов, можно собирать агрегацию по пользователям:

In [ ]:
review_facts = spark.table("golden_review_facts")
users = spark.table("users")

Например, можно число отзывов для пользователей, чей идентификатор оканчивается на `1412`:

In [ ]:
users.where("id % 10000 = 1412").show()

In [ ]:
reviews_users = (
    review_facts.alias("f")
        .join(
            users.alias("u"),
            col("f.user_id") == col("u.id")
        )
        .where("u.id % 10000 = 1412")
        .groupBy("u.id", "u.username")
        .count()
        .orderBy("count")
)
reviews_users.show()

Если в источники добавить новые отзывы, то знчения автоматически пересчитаются:

In [ ]:
reviews_metacritic = generate_reviews_metacritic(ids_range=range(30, 40))
reviews_igromania = generate_reviews_igromania(ids_range=range(21, 25))

In [ ]:
save_df(reviews_metacritic, f"{source_path}/reviews_metacritic", format="json", mode='append')
save_df(reviews_igromania, f"{source_path}/reviews_igromania", format="json", mode='append')

In [ ]:
reviews_users.show()

## Внешний анализ данных на Apache Superset

Для демонстрации возможностей нашей Lakehouse платформы подключим внешнюю систему для анализа и визуализации данных [Apache Superset](https://superset.apache.org/).

Apache Superset умеет подключать большое число внешних баз данных, но нас интересует возможностей построения отчетов на базе Delta Lake файлов в нашем озере данных.

Apache Superset имеет поддержку базы данных [Trino](https://trino.io), которая умеет [читать](https://trino.io/docs/current/connector/delta-lake.html) Delta Lake файлы из HDFS.

### Конфигурация Apache Superset

In [ ]:
# ! source ~/.bash_aliases && HOST=dind execute \
# docker compose --profile=trino --profile=superset up -d trino superset

In [ ]:
! source ~/.bash_aliases && HOST=dind execute \
docker compose ps trino superset

In [ ]:
! source ~/.bash_aliases && HOST=superset execute \
pip install trino && \
docker compose restart superset

Нужно дождаться, когда в колонке STATUS появится будет значение `healthy`:

In [ ]:
! source ~/.bash_aliases && HOST=dind execute \
docker compose ps trino superset | hl 'superset\|trino\|healthy'

Необходимо создать пользователя:

In [ ]:
! source ~/.bash_aliases && HOST=superset execute \
superset fab create-admin \
              --username admin \
              --password admin \
              --firstname Superset \
              --lastname Admin \
              --email 'admin@superset.com'

Создать объекты Apache Superset в базе данных:

In [ ]:
! source ~/.bash_aliases && HOST=superset execute \
superset db upgrade

Запустить инициализацию Apache Superset:

In [ ]:
! source ~/.bash_aliases && HOST=superset execute \
superset init

### Настройка подключения к Trino

In [ ]:
import requests
import json

Создать токен для работы через REST API:

In [ ]:
payload = {
  "username": "admin",
  "password": "admin",
  "provider": "db"
}
r = requests.post('http://superset:8088/api/v1/security/login', json=payload)

assert r.status_code == 200

access_token = r.json()["access_token"]

Создать CSRF токен для работы через REST API:

In [ ]:
headers = {'Authorization': f'Bearer {access_token}'}
r = requests.get('http://superset:8088/api/v1/security/csrf_token/', headers=headers)

assert r.status_code == 200

csrf = r.json()["result"]
cookies = r.cookies

In [ ]:
headers = {
    "Authorization": f"Bearer {access_token}",
    "X-CSRFToken": csrf
}

Зарегистрировать Trino:

In [ ]:
payload = {
    "database_name": "Trino Lakehouse",
    "sqlalchemy_uri": "trino://trino@trino:8080/delta_lake",
    "engine": "trino"
}

r = requests.post('http://superset:8088/api/v1/database', headers=headers, json=payload, cookies=cookies)

assert r.status_code == 201

r.json()

In [ ]:
database_id = r.json()["id"]

Зарегистрировать таблицы:

In [ ]:
payload = {
    "database": database_id,
    "schema": "default"
}

for table in ["users", "bronze_reviews_igromania", "bronze_reviews_metacritic", "silver_reviews", "golden_review_facts"]:
    print(f"Registering {table}")
    payload["table_name"] = table
    
    r = requests.post('http://superset:8088/api/v1/dataset', headers=headers, json=payload, cookies=cookies)
    if r.status_code != 201:
        print(r.json())
        assert False

Зарегистрировать датасет на базе таблицы `golden_review_facts`:

In [ ]:
payload = {
    "database": database_id,
    "schema": "default",
    "table_name": "facts_enriched",
    "is_managed_externally": False,
    "external_url": None,
    "sql": """
SELECT review_id
     , r.app_id
     , f.user_id
     , username
     , gender
     , dob
     , title
     , date_release
     , win
     , mac
     , linux
     , price_final
  FROM golden_review_facts f
  JOIN users u ON (u.id = f.user_id)
  JOIN games g USING (app_id)
  JOIN silver_reviews r ON (r.id = f.review_id)
"""
}
r = requests.post('http://superset:8088/api/v1/dataset', headers=headers, json=payload, cookies=cookies)

assert r.status_code == 201

In [ ]:
datasource_id = r.json()["id"]
datasource_uid = r.json()["data"]["uid"]

Создать график, который покажет по годам сколько отзывов оставляли мужчины, а сколько женщины:

In [ ]:
payload = {
    "params": f"""{{
        "datasource": "{datasource_uid}",
        "viz_type": "echarts_timeseries_bar",
        "x_axis": "date_release",
        "time_grain_sqla": "P1Y",
        "x_axis_sort_asc": true,
        "x_axis_sort_series": "name",
        "x_axis_sort_series_ascending": true,
        "metrics": [
            {{
                "expressionType": "SIMPLE",
                "column": {{
                    "advanced_data_type": null,
                    "certification_details": null,
                    "certified_by": null,
                    "column_name": "review_id",
                    "description": null,
                    "expression": null,
                    "filterable": true,
                    "groupby": true,
                    "id": 20,
                    "is_certified": false,
                    "is_dttm": false,
                    "python_date_format": null,
                    "type": "VARCHAR",
                    "type_generic": 1,
                    "verbose_name": null,
                    "warning_markdown": null
                }},
                "aggregate": "COUNT_DISTINCT",
                "sqlExpression": null,
                "datasourceWarning": false,
                "hasCustomLabel": false,
                "label": "COUNT_DISTINCT(review_id)",
                "optionName": "metric_x2g7cq064tb_vpz0jzxjivl"
            }}
        ],
        "groupby": [
            "gender"
        ],
        "adhoc_filters": [
            {{
                "expressionType": "SIMPLE",
                "subject": "date_release",
                "operator": "TEMPORAL_RANGE",
                "comparator": "No filter",
                "clause": "WHERE",
                "sqlExpression": null,
                "isExtra": false,
                "isNew": false,
                "datasourceWarning": false,
                "filterOptionName": "filter_9glfwpxhc85_loni87wp65"
            }}
        ],
        "order_desc": true,
        "row_limit": 50000,
        "truncate_metric": true,
        "show_empty_columns": true,
        "comparison_type": "values",
        "annotation_layers": [],
        "forecastPeriods": 10,
        "forecastInterval": 0.8,
        "orientation": "vertical",
        "x_axis_title_margin": 15,
        "y_axis_title_margin": 15,
        "y_axis_title_position": "Left",
        "sort_series_type": "sum",
        "color_scheme": "supersetColors",
        "only_total": true,
        "show_legend": true,
        "legendType": "scroll",
        "legendOrientation": "top",
        "x_axis_time_format": "smart_date",
        "y_axis_format": "SMART_NUMBER",
        "y_axis_bounds": [
            null,
            null
        ],
        "rich_tooltip": true,
        "tooltipTimeFormat": "smart_date",
        "extra_form_data": {{}},
        "dashboards": []
    }}
    """,
    "slice_name": "Reviews by Gender",
    "viz_type": "echarts_timeseries_bar",
    "datasource_id": datasource_id,
    "datasource_type": "table",
    "dashboards": [],
    "query_context": """{
        "datasource": {
            "id": 6,
            "type": "table"
        },
        "force": false,
        "queries": [
            {
                "filters": [
                    {
                        "col": "date_release",
                        "op": "TEMPORAL_RANGE",
                        "val": "No filter"
                    }
                ],
                "extras": {
                    "having": "",
                    "where": ""
                },
                "applied_time_extras": {},
                "columns": [
                    {
                        "timeGrain": "P1Y",
                        "columnType": "BASE_AXIS",
                        "sqlExpression": "date_release",
                        "label": "date_release",
                        "expressionType": "SQL"
                    },
                    "gender"
                ],
                "metrics": [
                    {
                        "expressionType": "SIMPLE",
                        "column": {
                            "advanced_data_type": null,
                            "certification_details": null,
                            "certified_by": null,
                            "column_name": "review_id",
                            "description": null,
                            "expression": null,
                            "filterable": true,
                            "groupby": true,
                            "id": 20,
                            "is_certified": false,
                            "is_dttm": false,
                            "python_date_format": null,
                            "type": "VARCHAR",
                            "type_generic": 1,
                            "verbose_name": null,
                            "warning_markdown": null
                        },
                        "aggregate": "COUNT_DISTINCT",
                        "sqlExpression": null,
                        "datasourceWarning": false,
                        "hasCustomLabel": false,
                        "label": "COUNT_DISTINCT(review_id)",
                        "optionName": "metric_x2g7cq064tb_vpz0jzxjivl"
                    }
                ],
                "orderby": [
                    [
                        {
                            "expressionType": "SIMPLE",
                            "column": {
                                "advanced_data_type": null,
                                "certification_details": null,
                                "certified_by": null,
                                "column_name": "review_id",
                                "description": null,
                                "expression": null,
                                "filterable": true,
                                "groupby": true,
                                "id": 20,
                                "is_certified": false,
                                "is_dttm": false,
                                "python_date_format": null,
                                "type": "VARCHAR",
                                "type_generic": 1,
                                "verbose_name": null,
                                "warning_markdown": null
                            },
                            "aggregate": "COUNT_DISTINCT",
                            "sqlExpression": null,
                            "datasourceWarning": false,
                            "hasCustomLabel": false,
                            "label": "COUNT_DISTINCT(review_id)",
                            "optionName": "metric_x2g7cq064tb_vpz0jzxjivl"
                        },
                        false
                    ]
                ],
                "annotation_layers": [],
                "row_limit": 250,
                "series_columns": [
                    "gender"
                ],
                "series_limit": 0,
                "order_desc": true,
                "url_params": {},
                "custom_params": {},
                "custom_form_data": {},
                "time_offsets": [],
                "post_processing": [
                    {
                        "operation": "pivot",
                        "options": {
                            "index": [
                                "date_release"
                            ],
                            "columns": [
                                "gender"
                            ],
                            "aggregates": {
                                "COUNT_DISTINCT(review_id)": {
                                    "operator": "mean"
                                }
                            },
                            "drop_missing_columns": false
                        }
                    },
                    {
                        "operation": "rename",
                        "options": {
                            "columns": {
                                "COUNT_DISTINCT(review_id)": null
                            },
                            "level": 0,
                            "inplace": true
                        }
                    },
                    {
                        "operation": "flatten"
                    }
                ]
            }
        ],
        "form_data": {
            "datasource": "6__table",
            "viz_type": "echarts_timeseries_bar",
            "x_axis": "date_release",
            "time_grain_sqla": "P1Y",
            "x_axis_sort_asc": true,
            "x_axis_sort_series": "name",
            "x_axis_sort_series_ascending": true,
            "metrics": [
                {
                    "expressionType": "SIMPLE",
                    "column": {
                        "advanced_data_type": null,
                        "certification_details": null,
                        "certified_by": null,
                        "column_name": "review_id",
                        "description": null,
                        "expression": null,
                        "filterable": true,
                        "groupby": true,
                        "id": 20,
                        "is_certified": false,
                        "is_dttm": false,
                        "python_date_format": null,
                        "type": "VARCHAR",
                        "type_generic": 1,
                        "verbose_name": null,
                        "warning_markdown": null
                    },
                    "aggregate": "COUNT_DISTINCT",
                    "sqlExpression": null,
                    "datasourceWarning": false,
                    "hasCustomLabel": false,
                    "label": "COUNT_DISTINCT(review_id)",
                    "optionName": "metric_x2g7cq064tb_vpz0jzxjivl"
                }
            ],
            "groupby": [
                "gender"
            ],
            "adhoc_filters": [
                {
                    "expressionType": "SIMPLE",
                    "subject": "date_release",
                    "operator": "TEMPORAL_RANGE",
                    "comparator": "No filter",
                    "clause": "WHERE",
                    "sqlExpression": null,
                    "isExtra": false,
                    "isNew": false,
                    "datasourceWarning": false,
                    "filterOptionName": "filter_9glfwpxhc85_loni87wp65"
                }
            ],
            "order_desc": true,
            "row_limit": 50000,
            "truncate_metric": true,
            "show_empty_columns": true,
            "comparison_type": "values",
            "annotation_layers": [],
            "forecastPeriods": 10,
            "forecastInterval": 0.8,
            "orientation": "vertical",
            "x_axis_title_margin": 15,
            "y_axis_title_margin": 15,
            "y_axis_title_position": "Left",
            "sort_series_type": "sum",
            "color_scheme": "supersetColors",
            "only_total": true,
            "show_legend": true,
            "legendType": "scroll",
            "legendOrientation": "top",
            "x_axis_time_format": "smart_date",
            "y_axis_format": "SMART_NUMBER",
            "y_axis_bounds": [
                null,
                null
            ],
            "rich_tooltip": true,
            "tooltipTimeFormat": "smart_date",
            "extra_form_data": {},
            "dashboards": [],
            "force": false,
            "result_format": "json",
            "result_type": "full"
        },
        "result_format": "json",
        "result_type": "full"
    }
    """
}

In [ ]:
r = requests.post('http://superset:8088/api/v1/chart', headers=headers, json=payload, cookies=cookies)
assert r.status_code == 201

Apache Superset готов к работе. Apache Superset запущен на 18088 порту, веб-интерфейс Apache Superset можно найти по ссылке [http://localhost:18088](http://localhost:18088).

### Аналитика в реальном времени

Для демонстрации аналитики в реальном времени выполните:

1. Создайте новый [дашборд](http://localhost:18088/dashboard/list/) (кнопка `+ DASHBOARD` в правом верхнем углу) с именем `[ Lakehouse ]`;
1. Добавьте на дашборд `[ Lakehouse ]` график `Reviews by Gender` (перетащить график мышью с правой панели в центр);
1. Нажмите `SAVE` в правом верхнем углу.

4. Добавьте новые отзывы:

In [ ]:
reviews_metacritic = generate_reviews_metacritic(ids_range=range(30, 40))
reviews_igromania = generate_reviews_igromania(ids_range=range(21, 25))

In [ ]:
save_df(reviews_metacritic, f"{source_path}/reviews_metacritic", format="json", mode='append')
save_df(reviews_igromania, f"{source_path}/reviews_igromania", format="json", mode='append')

5. Откройте дашборд [`[ Lakehouse ]`](http://localhost:18088/dashboard/list/1)
6. Обновляйте график (кнопка `...` в правом верхнем углу -> `Refresh dashboard`) и следите как меняются числа.

In [ ]:
golden_review_facts_stream.awaitTermination(timeout=5)
golden_review_facts_stream.stop()

In [ ]:
silver_reviews_from_metacritic_stream.awaitTermination(timeout=5)
silver_reviews_from_metacritic_stream.stop()

In [ ]:
silver_reviews_from_igromania_stream.awaitTermination(timeout=5)
silver_reviews_from_igromania_stream.stop()

In [ ]:
bronze_reviews_igromania_stream.awaitTermination(timeout=5)
bronze_reviews_igromania_stream.stop()

In [ ]:
bronze_reviews_metacritic_stream.awaitTermination(timeout=5)
bronze_reviews_metacritic_stream.stop()

In [ ]:
! source ~/.bash_aliases && \
docker compose stop trino superset

## Машинное обучение

Платформа Lakehouse позволяет построить платформу данных для задач машинного обучения в том числе. Catboost поддерживает работу с Apache Spark.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col
from pyspark.sql import functions as F

In [ ]:
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages ai.catboost:catboost-spark_3.4_2.12:1.2.2,io.delta:delta-spark_2.12:3.0.0 pyspark-shell'

In [ ]:
builder = (
    SparkSession
        .builder
        .appName("Catboost Lakehouse")
        .master("local[4]")
        .config("spark.sql.warehouse.dir", "data/spark-warehouse")
        .config("hive.metastore.uris", "thrift://hive:9083")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .enableHiveSupport()
)

spark = configure_spark_with_delta_pip(builder) \
    .getOrCreate()

sc = spark.sparkContext

In [ ]:
import catboost_spark

In [ ]:
from pyspark.sql import Row,SparkSession
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import *

In [ ]:
games = spark.table("games")
users = spark.table("users")
reviews = spark.table("silver_reviews")
review_facts_df = spark.table("golden_review_facts")

In [ ]:
games.printSchema()

In [ ]:
train_df = (
    review_facts_df.alias("f")
        .join(games, "app_id")
        .join(users.alias("u"), col("f.user_id") == col("u.id"))
        .join(reviews.alias("r"), col("f.review_id") == col("r.id"))
)

train_pool = catboost_spark.Pool(train_df)

In [ ]:
spark.stop()

## Вывод

Формат данных Delta Lake в совокупности с Apache Spark Structured Streaming позволяет по шаблону Lakehouse построить надежную платформу для аналитики данных в реальном времени, что повышает устойчивость бизнеса к резким колебаниям на рынке. При этом Structured Streaming работает в отрыве от Apache Kafka или любого другого брокера сообщений, что повышает требования к эксплуатации в части мониторинга. С другой стороны Structured Streaming в случае падения воркера Apache Spark перезапустит его автоматически и стриминг данных продолжится. Это, конечно, избавляет от аппартных проблем, но если воркер падает по `OutOfMemoryError`, то воркеры будут падать и перезапускаться постоянно, поэтому нужно внимательно работать с памятью при настройке стримов.